Churn prediction
-------------------
Company churn occurs when companies cancel subscriptions or subscriptions expires without renewals.  
This file uses LogisticRegression to predict Churn. It also uses SMOTE for imbalanced data, and RFE for feature selection.

In [2]:
%run /Users/bxing@actsoft.com/churn_dataframe_sandbox

In [3]:
print(df.shape)
print(list(df.columns))

(7240, 20)
['CompanyId', 'CompanyName', 'Tier', 'IsSetupComplete', 'UserIntegrationType', 'City', 'RegionName', 'PostalCode', 'CountryCode', 'CreationDate', 'ModificationDate', 'CarrierId', 'Culture', 'DynamicsGuid', 'Licenses', 'Activation', 'Expiration', 'Deactivation', 'Duration', 'Churn']

In [4]:
df = df[categorical_features + [target] + numerical_features]
print(list(df.columns))

['Tier', 'IsSetupComplete', 'UserIntegrationType', 'Churn', 'Licenses', 'Duration']

In [5]:
df.groupby('Tier').mean()

,IsSetupComplete,UserIntegrationType,Churn,Licenses,Duration
Tier,,,,,
5,0.777778,0.000000,0.555556,92.000000,442.866667
10,0.357090,0.002204,0.525839,224.385746,322.771736
20,0.563656,0.000000,0.596300,37.048966,343.851469
30,0.492151,0.002355,0.589482,51.147567,362.547096


In [6]:
df.groupby('IsSetupComplete').mean()

,Tier,UserIntegrationType,Churn,Licenses,Duration
IsSetupComplete,,,,,
False,15.11998,0.001469,0.525465,31.370225,292.672380
True,17.20057,0.001901,0.593473,293.234791,391.766793


In [7]:
df.groupby('UserIntegrationType').mean()

,Tier,IsSetupComplete,Churn,Licenses,Duration
UserIntegrationType,,,,,
0,16.027501,0.435876,0.555003,145.52225,335.800304
3,15.000000,0.500000,0.750000,141.50000,459.750000


In [8]:
count_no_churn = len(df[df['Churn']==0])
count_churn = len(df[df['Churn']==1])
pct_of_no_churn = count_no_churn/(count_no_churn+count_churn)
print("percentage of no churn is", pct_of_no_churn*100)
pct_of_churn = count_churn/(count_no_churn+count_churn)
print("percentage of churn", pct_of_churn*100)

percentage of no churn is 44.488950276243095
percentage of churn 55.51104972375691

Create dummy variables for categorical variables

In [10]:
cat_vars=['Tier','IsSetupComplete','UserIntegrationType']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var)
    df1=df.join(cat_list)
    df=df1
    
df_vars=df.columns.values.tolist()
to_keep=[i for i in df_vars if i not in cat_vars]
df_final=df[to_keep]
df_final.columns.values

Out[233]: array(['Churn', 'Licenses', 'Duration', 'Tier_5', 'Tier_10', 'Tier_20',
 'Tier_30', 'IsSetupComplete_False', 'IsSetupComplete_True',
 'UserIntegrationType_0', 'UserIntegrationType_3'], dtype=object)

In [11]:
df_final.head()

,Churn,Licenses,Duration,Tier_5,Tier_10,Tier_20,Tier_30,IsSetupComplete_False,IsSetupComplete_True,UserIntegrationType_0,UserIntegrationType_3
0,0,28,1077,0,0,1,0,0,1,1,0
1,1,6,191,0,0,1,0,0,1,1,0
2,1,13,219,0,0,0,1,0,1,1,0
3,0,7,935,0,1,0,0,0,1,1,0
4,1,10,92,0,0,0,1,0,1,1,0


Let's scale the data before doing RFE and Fitting.

In [13]:
from sklearn.preprocessing import StandardScaler

scaled_features = StandardScaler().fit_transform(df_final.values[:, [1,2]])
scaled_features_df = pd.DataFrame(scaled_features, index=df_final.index, columns=['Licenses', 'Duration'])

# df_final['Licenses'] = scaled_features_df['Licenses']
# df_final['Duration'] = scaled_features_df['Duration']
df_final.update(scaled_features_df)

df_final.head()

,Churn,Licenses,Duration,Tier_5,Tier_10,Tier_20,Tier_30,IsSetupComplete_False,IsSetupComplete_True,UserIntegrationType_0,UserIntegrationType_3
0,0,-0.046574,2.770786,0,0,1,0,0,1,1,0
1,1,-0.055293,-0.541605,0,0,1,0,0,1,1,0
2,1,-0.052518,-0.436925,0,0,0,1,0,1,1,0
3,0,-0.054896,2.239907,0,1,0,0,0,1,1,0
4,1,-0.053707,-0.911726,0,0,0,1,0,1,1,0


Data looks like unbalanced. Let's perform **Over-sampling using SMOTE**.

In [15]:
from sklearn import preprocessing

X = df_final.loc[:, df_final.columns != 'Churn']
y = df_final.loc[:, df_final.columns == 'Churn']

# whether over_sampling or not
sampling = True

In [16]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

if sampling:
  os_data_X,os_data_y=os.fit_sample(X_train, y_train)
  os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
  os_data_y= pd.DataFrame(data=os_data_y,columns=['Churn'])
else:
  os_data_X = X_train
  os_data_y = y_train

# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no churn in oversampled data",len(os_data_y[os_data_y['Churn']==0]))
print("Number of churn",len(os_data_y[os_data_y['Churn']==1]))
print("Proportion of no churn data in oversampled data is ",len(os_data_y[os_data_y['Churn']==0])/len(os_data_X))
print("Proportion of churn data in oversampled data is ",len(os_data_y[os_data_y['Churn']==1])/len(os_data_X))

length of oversampled data is 5644
Number of no churn in oversampled data 2822
Number of churn 2822
Proportion of no churn data in oversampled data is 0.5
Proportion of churn data in oversampled data is 0.5

Recursive feature elimination

In [18]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

rfe = RFE(logreg, n_features_to_select=20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

[ True True True True True True True True True True]
[1 1 1 1 1 1 1 1 1 1]

RFE doesn't rule out any features. So we keep all.

In [20]:
X=os_data_X
y=os_data_y['Churn']

print(X.columns.values)

['Licenses' 'Duration' 'Tier_5' 'Tier_10' 'Tier_20' 'Tier_30'
 'IsSetupComplete_False' 'IsSetupComplete_True' 'UserIntegrationType_0'
 'UserIntegrationType_3']

Implement the model. Using Logit to analyze p-values, and remove any columns that have high p-values.  
In this case, it yields overflow error. So we proceed with all original columns.

In [22]:
import statsmodels.api as sm

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Warning: Maximum number of iterations has been exceeded.
 Current function value: 0.610473
 Iterations: 35
/databricks/python/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
 "Check mle_retvals", ConvergenceWarning)
/databricks/python/lib/python3.7/site-packages/statsmodels/base/model.py:1354: RuntimeWarning: invalid value encountered in sqrt
 bse_ = np.sqrt(np.diag(self.cov_params()))
/databricks/python/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
 return (a < x) & (x < b)
/databricks/python/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
 return (a < x) & (x < b)
/databricks/python/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
 cond2 = cond0 & (x <= _a)
 Results: Logit
======================================================================================
Model: Logit Pseudo R-squared: 0.119 
Dependent Variable: Churn AIC: 6909.0210 
Date: 2020-07-13 00:23 BIC: 6968.7661 
No. Observations: 5644 Log-Likelihood: -3445.5 
Df Model: 8 LL-Null: -3912.1 
Df Residuals: 5635 LLR p-value: 3.8406e-196
Converged: 0.0000 Scale: 1.0000 
No. Iterations: 35.0000 
--------------------------------------------------------------------------------------
 Coef. Std.Err. z P>|z| [0.025 0.975] 
--------------------------------------------------------------------------------------
Licenses -8.6396 1.3324 -6.4843 0.0000 -11.2510 -6.0281
Duration -0.8018 0.0327 -24.5082 0.0000 -0.8659 -0.7377
Tier_5 28.2934 988174.8259 0.0000 1.0000 -1936758.7758 1936815.3626
Tier_10 28.1045 988174.8259 0.0000 1.0000 -1936758.9647 1936815.1737
Tier_20 28.3756 988174.8259 0.0000 1.0000 -1936758.6936 1936815.4448
Tier_30 28.4549 988174.8259 0.0000 1.0000 -1936758.6143 1936815.5241
IsSetupComplete_False -6.8920 nan nan nan nan nan
IsSetupComplete_True -6.3154 nan nan nan nan nan
UserIntegrationType_0 -21.9563 nan nan nan nan nan
UserIntegrationType_3 8.7490 3058944.7473 0.0000 1.0000 -5995412.7864 5995430.2844
======================================================================================

Logistic Regression Model Fitting

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Out[241]: LogisticRegression()

In [25]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.65

**Testing**
- Confusion Matrix
- Classification Report
- AOC Curve

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[521 306]
 [281 586]]

In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

precision recall f1-score support

 0 0.65 0.63 0.64 827
 1 0.66 0.68 0.67 867

 accuracy 0.65 1694
 macro avg 0.65 0.65 0.65 1694
weighted avg 0.65 0.65 0.65 1694

In [29]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()